### Predict RA & Dec for an asteroid on three nights in the summer

In [1]:
'''
download DAT file from https://www.minorplanetcenter.net/data
grep -E '^00001|^00002|^00003' MPCORB.DAT > MPCORB.excerpt.DAT
grep -E '2012 FN62' MPCORB.DAT > MPCORB.2012FN62.DAT
'''
import pandas as pd
import numpy as np
from skyfield.data import mpc
from skyfield.api import load, N, W, wgs84
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN

# Load Sun Earth
eph = load('MPC/de421.bsp')
sun, earth = eph['sun'], eph['earth']
# Site
obs_site = earth + wgs84.latlon(35.904613 * N, 79.046761 * W)
# Asteroid
target = '2012FN62'

# Load orbital elements
with load.open(f'MPC/MPCORB.{target}.DAT') as f:
    minor_planets = mpc.load_mpcorb_dataframe(f)
row = minor_planets.loc[0].copy() # Create an explicit copy

# Compute orbit
ts = load.timescale()
asteroid = sun + mpc.mpcorb_orbit(row, ts, GM_SUN)

# Observing times, use TT (terrestrial timescale) because 
# in skyfield/data/mpc.py, mpcorb_orbit(), t_epoch = ts.tt_jd(epoch_jd)
# Timescale.tt:	Build a Time Object from a Terrestrial Time Calendar date.
t0 = ts.tt(2025, 5, 1) # starting calendar date
# Time.tt:	Terrestrial Time (TT) as a Julian date.
jd0 = t0.tt
# Timescale.tt_jd: Build a Time object from a Terrestrial Time Julian date.
ndata = 3 
nspan = 150 
jds = np.linspace(jd0,jd0+nspan,ndata)
# dates from data2012FN62.txt as a sanity check
#jds = np.array([2460480.708333333,2460487.708333333,2460494.708333333])
t = ts.tt_jd(jds) # time objects from JDs

# astrometric RA & Dec (matching astrometric RA & DEC from JPL Horizon)
ra, dec, distance = earth.at(t).observe(asteroid).radec()
cat = pd.DataFrame({'JD':jds})
cat['RA'] = ra._degrees
cat['Dec'] = dec._degrees

# Sun vector, dim (3, ntime)
Rs = earth.at(t).observe(sun).xyz.au
cat['Rx'] = Rs[0,:]
cat['Ry'] = Rs[1,:]
cat['Rz'] = Rs[2,:]

# Total positional error
error_deg = 1 / 3600.0  # 1 arcsec = 1/3600 deg
sigma = error_deg / np.sqrt(2)  # Standard deviation per component
# Define the uncorrelated covariance matrix
# Note: sigma is the stddev of the marginalized pdf in each axis
cov = [[sigma**2, 0],
       [0, sigma**2]]
# Generate bivariate Gaussian noise
np.random.seed(42)
noise = np.random.multivariate_normal([0, 0], cov, size=ndata)
# Extract RA and Dec errors
ra_errors = noise[:, 0] / np.cos(np.radians(cat.Dec))  # Scale RA errors by 1/cos(Dec)
dec_errors = noise[:, 1]
# Add errors to RA and Dec
cat['RA_obs'] = cat['RA'] + ra_errors
cat['Dec_obs'] = cat['Dec'] + dec_errors
cat['perr'] = np.zeros(ndata) + error_deg 

# save to CSV file
cat.to_csv(f'sim_data/{target}_s{nspan}_n{ndata}.csv')
cat

,JD,RA,Dec,Rx,Ry,Rz,RA_obs,Dec_obs,perr
0,2460796.5,49.497988,10.456323,0.765771,0.600715,0.260398,49.498087,10.456296,0.000278
1,2460871.5,73.457877,14.282440,-0.387808,0.862005,0.373666,73.458008,14.282739,0.000278
2,2460946.5,86.143607,12.875771,-0.998653,-0.076572,-0.033190,86.143559,12.875725,0.000278


### Compare with JPL Horizon

In [2]:
from astroquery.jplhorizons import Horizons
from astropy.time import Time
import astropy.units as u

# Observation site
UNC = {'lon': -79.046761 * u.deg,
        'lat': 35.904613 * u.deg,
        'elevation': 0.152 * u.km}

# Observation time
#obs_time = Time('2025-05-01T00:00:00', format='isot', scale='utc')
#epochs = obs_time.jd
epochs = jds

# Asteroid Target
target = '2012 FN62'

# 500@10: heliocentric elements 
obj = Horizons(id=target, location='500@10', epochs=epochs, id_type='smallbody')
elem = obj.elements()
# 500@399: geocentric location for ephemeris
obj = Horizons(id=target, location='500@399', epochs=epochs, id_type='smallbody')
ephe = obj.ephemerides()

# Extract relevant elements
jpl_elms = {
    'a': elem['a'][0],  # Semi-major axis (AU)
    'e': elem['e'][0],  # Eccentricity
    'incl': elem['incl'][0],  # Inclination (degrees)
    'Omega': elem['Omega'][0],  # Longitude of ascending node (degrees)
    'w': elem['w'][0],  # Argument of perihelion (degrees)
    'MA': elem['M'][0],  # Mean anomaly (degrees)
    'epoch_jd': elem['datetime_jd'][0],  # Epoch (JD)
    'Tp_jd': elem['Tp_jd'][0], # Time of Perihelion passage (JD)
    'q': elem['q'][0],  # Perihelion distance (AU)
    'Q': elem['Q'][0],  # Aphelion distance (AU)
    'P': elem['P'][0],  # Orbital period (days)
}

ephe[['datetime_jd','RA','DEC','RA_app','DEC_app']]

datetime_jd,RA,DEC,RA_app,DEC_app
d,deg,deg,deg,deg
float64,float64,float64,float64,float64
2460796.5,49.49826,10.4564,49.83706,10.54802
2460871.5,73.45875,14.28275,73.81697,14.32523
2460946.5,86.14714,12.87721,86.51141,12.89004


In [3]:
cat

,JD,RA,Dec,Rx,Ry,Rz,RA_obs,Dec_obs,perr
0,2460796.5,49.497988,10.456323,0.765771,0.600715,0.260398,49.498087,10.456296,0.000278
1,2460871.5,73.457877,14.282440,-0.387808,0.862005,0.373666,73.458008,14.282739,0.000278
2,2460946.5,86.143607,12.875771,-0.998653,-0.076572,-0.033190,86.143559,12.875725,0.000278
